In [3]:
#Declare packages
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
#df = pd.read_csv('C:/Users/HP/Pictures/HR/HR-Analytics/data/raw/HR-Employee-Attrition.csv')
df=pd.read_csv("/content/HR-Employee-Attrition.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# Exploration Data Analysis

Xem xét quy mô công ty

In [5]:
fig=make_subplots(rows=1, cols=2,
                  subplot_titles=("", "Số lượng nhân viên nghỉ việc theo phòng ban"),
                  specs=[[{"type": "bar"}, {"type": "pie"}]])

# Bar chart
plot_df=df['Attrition'].value_counts(normalize=True)
plot_df=plot_df.mul(100).rename('Percent').reset_index().sort_values('Percent')
plot_df.rename(columns={'index':'Attrition'}, inplace=True)
plot_df['Attrition']=['Cựu nhân viên' if i == 'Yes' else 'Nhân viên hiện tại' for i in plot_df['Attrition']]
x=plot_df['Attrition']
y=plot_df['Percent']
fig.add_trace(
    go.Bar(x=x, y=y, text=y,opacity=.8,
           hovertemplate='Employee Attrition Rate<br>%{x}: %{y:.3}%<extra></extra>',
           showlegend=False), row=1, col=1)
fig.update_traces(texttemplate='%{text:.3s}%', textposition='outside',
                  marker_line=dict(width=1, color='#1F0202'), marker_color=["#C41823","#368EED"])
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='gray')
fig.update_layout(yaxis_ticksuffix = '%')

# Pie chart
plot_df2=df[df.Attrition=='Yes']
plot_df2=plot_df2['Department'].value_counts(normalize=True)
plot_df2=plot_df2.mul(100).rename('Percent').reset_index().sort_values('Percent', ascending=False)
plot_df2.rename(columns={'index':'Department'}, inplace=True)
fig.add_trace(go.Pie(labels=plot_df2['Department'], values=plot_df2['Percent'], opacity=0.85, hole=0.4,
                     hovertemplate='%{label}<br>Attrition Rate: %{value:.3}%<extra></extra>',
                     marker_colors=["#A70C09","#E74824","#D08841"]), row=1, col=2)
                    #  marker_colors=['#587D65','#ADC4B2','#D1C9C2']), row=1, col=2)
fig.update_yaxes(tickmode = 'array', range=[0, 90], dtick=5)
fig.update_traces(textfont_size=14,textfont_color='black',marker=dict(line=dict(color='#28221D', width=1)))
fig.update_layout(title_text="Thống kê về tình trạng làm việc tại công ty", font_color='#28221D',
                  paper_bgcolor='#F4F2F0', plot_bgcolor='#F4F2F0')
fig.show()

- Dựa vào biểu đồ ta thấy trong bộ dữ liệu bao gồm 16.1% nhân viên đã nghỉ việc và có 83.9% nhân viên vẫn còn đang làm
- Trong đó
    + Ở phòng ban R&D, số lượng nhân viên nghỉ việc chiếm tỷ lệ cao nhất, khoảng 51.3%
    + Ngay sau đó là phòng ban Sales, chiếm khoảng 38.8% tỷ lệ nhân viên nghỉ việc
    + Phòng ban HR chiếm tỷ lệ ít nhất, với 5.06%

In [9]:
age_att=df.groupby(['Age','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts')
fig=px.line(age_att,x='Age',y='Counts',color='Attrition',width=1000,labels={"Age":"Tuổi","Counts":"Số lượng", "Attrition":"Tình trạng nghỉ việc"},title='Số lượng nhân viên trong công ty theo độ tuổi ')
fig.for_each_trace(lambda trace: trace.update(name='Đang làm việc') if trace.name == 'No' else trace.update(name='Nghỉ việc'))
fig.show()

<ipython-input-9-26a72c40dfb6>:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



- Ta có thể thấy ở độ tuổi 26 đến 38 số lượng nhân viên nghỉ việc tăng cao. Điều này là vì đây là độ tuổi bắt đầu tìm kiếm việc làm nên họ thường có xu hướng muốn khám phá và tìm kiếm nhiều cơ hội mới, và họ yêu cầu trải nghiệm trong công việc nhiều hơn.
- Còn từ độ tuổi 40 trở về sau số lượng nhân viên nghỉ việc càng giảm. Bởi vì đây là độ tuổi mà họ cần có sự ổn định trong công việc để lo cho cuộc sống và đây cũng là độ tuổi có rủi ro tìm việc cao nhất

In [10]:
rate_att=df.groupby(['MonthlyIncome','Attrition']).apply(lambda x:x['MonthlyIncome'].count()).reset_index(name='Counts')
rate_att['MonthlyIncome']=round(rate_att['MonthlyIncome'],-3)
rate_att=rate_att.groupby(['MonthlyIncome','Attrition']).apply(lambda x:x['MonthlyIncome'].count()).reset_index(name='Counts')
fig=px.line(rate_att,x='MonthlyIncome',y='Counts',color='Attrition',width=1000,labels={"MonthlyIncome":"Thu nhập hàng tháng","Counts":"Số lượng","Attrition":"Tình trạng nghỉ việc"},title='Số lượng nhân viên theo thu nhập hàng tháng trong công ty   ')
fig.for_each_trace(lambda trace: trace.update(name='Đang làm việc') if trace.name == 'No' else trace.update(name='Nghỉ việc'))
fig.show()

<ipython-input-10-9e66880fd4a4>:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-10-9e66880fd4a4>:3: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



- Ta có thể thấy ở mức lương thấp (<5k) số lượng nhân viên nghỉ việc cao và mức lương càng cao thì số lượng nhân viên nghỉ việc càng thấp
- Tuy nhiên ở mức lương 10k có sự gia tăng đột ngột, điều này có thể là vì họ đã có mức sống tốt hơn và muốn tìm một công việc khác để có tìm cơ hội tốt hơn

In [6]:
plot_df = df.groupby('JobRole')['MonthlyIncome'].mean()
plot_df = plot_df.mul(12).rename('Salary').reset_index().sort_values('Salary', ascending=False)
fig = px.bar(plot_df, x='JobRole', y='Salary', text='Salary', opacity=0.7)
fig.update_traces(texttemplate='$%{text:,.0f}', textposition='outside',
                  marker_line=dict(width=1, color='#28221D'), marker_color="#3497DE")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='#28221D')
fig.update_layout(title_text='Lương trung bình theo vai trò trong công ty', font_color='#28221D',
                  yaxis=dict(title='Salary',tickprefix='$'), height=500,
                  xaxis_title='', paper_bgcolor='#F4F2F0', plot_bgcolor='#F4F2F0')
fig.show()

Nhận xét: Ta thấy những người giữ chức vụ quan trọng trong công ty sẽ có những mức đãi ngộ trung bình cao hơn (đối với vị trí Manager là >200k, Reseach Director là >190k), ngoài ra những vị trí khác cũng có mức lương tương đối cao (Healthcare Representative viws ~90k, Manufacturing Director với ~87k, Sales Executive với ~ 83k)

Thông thường trong một số công ty, nhân viên sẽ giữ một phần cổ phiếu của công ty, và trong bộ dữ liệu này ta có trường 'StockOptionLevel' thể hiện sự sở hữu cổ phiếu của công ty của nhân viên theo cấp bậc

In [11]:
dept_sales = df[df['Department'] == 'Sales'].groupby('Attrition').apply(lambda x: x['DailyRate'].count()).reset_index(name='Counts')
dept_rnd = df[df['Department'] == 'Research & Development'].groupby('Attrition').apply(lambda x: x['DailyRate'].count()).reset_index(name='Counts')
dept_hr = df[df['Department'] == 'Human Resources'].groupby('Attrition').apply(lambda x: x['DailyRate'].count()).reset_index(name='Counts')
fig = make_subplots(rows=1, cols=3,
                    specs=[[{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}]],
                    subplot_titles=['Tỉ lệ nghỉ việc trong phòng Kinh doanh', 'Tỉ lệ nghỉ việc trong phòng Nghiên cứu và Phát triển', 'Tỉ lệ nghỉ việc trong phòng Nhân sự'])
fig.add_trace(
    px.pie(dept_sales, names='Attrition', values='Counts',labels={"Attrition":"Tình trạng nghỉ việc","Counts":"Số lượng"}).data[0],
    row=1, col=1
)

fig.add_trace(
    px.pie(dept_rnd, names='Attrition', values='Counts',labels={"Attrition":"Tình trạng nghỉ việc","Counts":"Số lượng"}).data[0],
    row=1, col=2
)

fig.add_trace(
    px.pie(dept_hr, names='Attrition', values='Counts',labels={"Attrition":"Tình trạng nghỉ việc","Counts":"Số lượng"}).data[0],
    row=1, col=3
)
fig.update_layout(
    title_text="Tỉ lệ nghỉ việc theo từng phòng ban",
    height=400, width=1400,
    showlegend=True
)
fig.update_traces(
    selector=dict(type='pie'),
    labels=['Đang làm việc' if label == 'No' else 'Nghỉ việc' for label in ['No', 'Yes']],marker=dict(colors=['#3497DE', '#DE1515']
))
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

<ipython-input-11-3c7c3605c520>:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-11-3c7c3605c520>:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

<ipython-input-11-3c7c3605c520>:3: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `i

Dựa vào các biểu đồ trên ta thấy:
- Phòng có tỉ lệ nghỉ việc nhiều nhất là phòng Kinh danh chiểm 20,6% trong tổng số 446 nhân viên.
- Kế tiếp là phòng Nhân sự chiếm 19% trong tổng số 63 nhân viên
- Và cuối cùng là phòng Nghiên cứu và Phát triển chiếm 13,8%  trong tổng số 961 nhân viên

In [7]:
stock_att=df.groupby(['StockOptionLevel','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts').replace({'No': 'Đang làm việc', 'Yes': 'Nghỉ việc'})
fig = px.bar(stock_att, x='StockOptionLevel', y='Counts', color='Attrition',
             title='Số lượng nhân viên nắm giữ cổ phiếu công ty theo phân cấp',
             width=1000,
             color_discrete_map={'Đang làm việc': '#3497DE', 'Nghỉ việc': "#DE1515"})

fig.update_layout(
    xaxis_title="Mức Sở Hữu Cổ Phiếu",
    yaxis_title="Số Lượng"
)

fig.update_traces(
    texttemplate='%{y:,.0f}',
    textposition='outside',
    marker_line=dict(width=1, color='#28221D')
)

fig.show()

<ipython-input-7-820f95c41940>:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



- Nhận xét:
Dựa vào biểu đồ, ta nhận thấy được đa số những nhân viên nghỉ việc thường nằm ở bậc 0 và 1, đây là những nhóm có sở hữu cổ phiếu bị hạn chế. Có thể thấy những người ít sở hữu cổ phần công ty có xu hướng nghỉ việc nhiều hơn.

In [12]:
sats_att=df.groupby(['EnvironmentSatisfaction','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts')
fig=px.area(sats_att,x='EnvironmentSatisfaction',y='Counts',color='Attrition',labels={"EnvironmentSatisfaction":"Mức độ hài lòng về môi trường làm việc","Counts":"Số lượng","Attrition":"Tình trạng nghỉ việc"},width=1200,title='Số lượng nhân viên theo mức độ hài lòng về nôi trường làm việc trong công ty')
fig.for_each_trace(lambda trace: trace.update(name='Đang làm việc') if trace.name == 'No' else trace.update(name='Nghỉ việc'))
fig.show()

<ipython-input-12-944ac9e8c74a>:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [13]:
jsats_att=df.groupby(['JobSatisfaction','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts')
fig=px.area(jsats_att,x='JobSatisfaction',y='Counts',color='Attrition',labels={"JobSatisfaction":"Mức độ hài lòng về công việc","Counts":"Số lượng","Attrition":"Tình trạng nghỉ việc"},width=1200,title='Số lượng nhân viên theo mức độ hài lòng về công việc trong công ty')
fig.for_each_trace(lambda trace: trace.update(name='Đang làm việc') if trace.name == 'No' else trace.update(name='Nghỉ việc'))
fig.show()

<ipython-input-13-a03d3992ab26>:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Liệu rằng yếu tố Cân bằng giữa Công việc có tác động đến tỉ lệ nghỉ việc của nhân viên tại không ty không, ta sẽ thực hiện một số trực quan để kiểm tra nhận định này

In [8]:
wlb_att=df.groupby(['WorkLifeBalance','Attrition']).apply(lambda x:x['DailyRate'].count()).reset_index(name='Counts').replace({'No': 'Đang làm việc', 'Yes': 'Nghỉ việc'})
fig = px.bar(wlb_att, x='WorkLifeBalance', y='Counts', color='Attrition',
             title='Biểu đồ thể hiện mức độ Cân bằng Công việc và Cuộc sống theo số lượng nhân viên',
             width=1000,
             color_discrete_map={'Đang làm việc': "#2792DE", 'Nghỉ việc': "#F30909"})

fig.update_layout(
    xaxis_title="Mức Cân Bằng Công Việc và Cuộc Sống",
    yaxis_title="Số Lượng"
)

fig.update_traces(
    texttemplate='%{y:,.0f}',
    textposition='outside',
    marker_line=dict(width=1, color='#28221D')
)

fig.show()

<ipython-input-8-9e09d894934b>:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



- Nhận xét:
Dựa vào biểu đồ ta thấy những người nằm ở mức thấp thường có xu hướng nghỉ việc, đối mới những người ở bậc cao hơn (bậc 3) xu hướng họ nghỉ việc nhiều hơn, có thể họ mong muốn tìm thêm cơ hội và đãi ngộ xứng đáng hơn, tuy nhiên biểu đồ này cho mức độ cân bằng cuộc sống ở mức 3 trở lên vẫn chiếm số lượng nhiều hơn, thấy nhân viên ở đây có sự hài lòng về công việc hiện tại ở công ty. Để kiểm tra nhận định này, ta sẽ tìm hiểu sự hài lòng của nhân viên đối với công ty như thế nào